<a href="https://colab.research.google.com/github/isak1997/Homeexam-3-STA-3001/blob/master/deep2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Version must be 1.16.1 to avoid pickle problems
!pip install numpy==1.16.1
import numpy as np

print(np.__version__)



1.16.1


In [0]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [0]:
%ls HE_data/

data/  DataLoaders/


In [0]:
#@title init { form-width: "5%" }

import torch as th
import torch.nn as nn
import matplotlib.pyplot as plt
import sys
from IPython.display import clear_output
from torch.utils.data import Dataset, DataLoader

device = 'cuda' if th.cuda.is_available() else 'cpu' 



In [0]:
sys.path.append('content/drive/My Drive/Colab Notebooks/HE_data/')
sys.path.append('/content/drive/My Drive/Colab Notebooks/HE_data/DataLoaders/')
datapath = sys.path.append('/content/drive/My Drive/Colab Notebooks/HE_data/data')

In [17]:
import camvid_loader 
Xtr_b, Ytr_b, Xte_b, Yte_b, class_desc = camvid_loader.load_camvid('/content/drive/My Drive/Colab Notebooks/HE_data/data/')



a = DataLoader(Xtr_b, batch_size=10, shuffle=True)   # Create iterator to loop through training data
b = DataLoader(Ytr_b, batch_size=10, shuffle=True)        # Create iterator to loop through test data.
c = DataLoader(Xte_b, batch_size=10, shuffle=True)   # Create iterator to loop through training data
d = DataLoader(Yte_b, batch_size=10, shuffle=True)        # Create iterator to loop through test data.

#xtr = th.tensor(a, dtype=th.float)
#ytr = th.tensor(b, dtype=th.float)
#xte = th.tensor(c, dtype=th.float)
#yte = th.tensor(d, dtype=th.float)




TypeError: ignored

In [0]:
#@title htrgtrg { form-width: "10px" }

class CNN(nn.Module):
    
    def __init__(self, n_class=21):
        super(CNN, self).__init__()
        # conv1
        self.conv1_1 = nn.Conv2d(3, 64, 3, padding=100)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, 3, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/2

        # conv2
        self.conv2_1 = nn.Conv2d(64, 128, 3, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, 3, padding=1)
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/4

        # conv3
        self.conv3_1 = nn.Conv2d(128, 256, 3, padding=1)
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/8

        # conv4
        self.conv4_1 = nn.Conv2d(256, 512, 3, padding=1)
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/16

        # conv5
        self.conv5_1 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/32

        # fc6
        self.fc6 = nn.Conv2d(512, 4096, 7)
        self.relu6 = nn.ReLU(inplace=True)
        self.drop6 = nn.Dropout2d()

        # fc7
        self.fc7 = nn.Conv2d(4096, 4096, 1)
        self.relu7 = nn.ReLU(inplace=True)
        self.drop7 = nn.Dropout2d()

        self.score_fr = nn.Conv2d(4096, n_class, 1)
        self.upscore = nn.ConvTranspose2d(n_class, n_class, 64, stride=32,
                                          bias=False)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
          if isinstance(m, nn.Conv2d):
            m.weight.data.zero_()
            if m.bias is not None:
              m.bias.data.zero_()
            if isinstance(m, nn.ConvTranspose2d):
              assert m.kernel_size[0] == m.kernel_size[1]
              initial_weight = get_upsampling_weight(
              m.in_channels, m.out_channels, m.kernel_size[0])
              m.weight.data.copy_(initial_weight)


    def forward(self, x):
          h = x
          h = self.relu1_1(self.conv1_1(h))
          h = self.relu1_2(self.conv1_2(h))
          h = self.pool1(h)

          h = self.relu2_1(self.conv2_1(h))
          h = self.relu2_2(self.conv2_2(h))
          h = self.pool2(h)

          h = self.relu3_1(self.conv3_1(h))
          h = self.relu3_2(self.conv3_2(h))
          h = self.relu3_3(self.conv3_3(h))
          h = self.pool3(h)

          h = self.relu4_1(self.conv4_1(h))
          h = self.relu4_2(self.conv4_2(h))
          h = self.relu4_3(self.conv4_3(h))
          h = self.pool4(h)

          h = self.relu5_1(self.conv5_1(h))
          h = self.relu5_2(self.conv5_2(h))
          h = self.relu5_3(self.conv5_3(h))
          h = self.pool5(h)

          h = self.relu6(self.fc6(h))
          h = self.drop6(h)

          h = self.relu7(self.fc7(h))
          h = self.drop7(h)

          h = self.score_fr(h)

          h = self.upscore(h)
          h = h[:, :, 19:19 + x.size()[2], 19:19 + x.size()[3]].contiguous()

          return h

    def copy_params_from_vgg16(self, vgg16):
      features = [
        self.conv1_1, self.relu1_1,
        self.conv1_2, self.relu1_2,
        self.pool1,
        self.conv2_1, self.relu2_1,
        self.conv2_2, self.relu2_2,
        self.pool2,
        self.conv3_1, self.relu3_1,
        self.conv3_2, self.relu3_2,
        self.conv3_3, self.relu3_3,
        self.pool3,
        self.conv4_1, self.relu4_1,
        self.conv4_2, self.relu4_2,
        self.conv4_3, self.relu4_3,
        self.pool4,
        self.conv5_1, self.relu5_1,
        self.conv5_2, self.relu5_2,
        self.conv5_3, self.relu5_3,
        self.pool5,
      ] 
      for l1, l2 in zip(vgg16.features, features):
          if isinstance(l1, nn.Conv2d) and isinstance(l2, nn.Conv2d):
              assert l1.weight.size() == l2.weight.size()
              assert l1.bias.size() == l2.bias.size()
              l2.weight.data = l1.weight.data
              l2.bias.data = l1.bias.data
      for i, name in zip([0, 3], ['fc6', 'fc7']):
          l1 = vgg16.classifier[i]
          l2 = getattr(self, name)
          l2.weight.data = l1.weight.data.view(l2.weight.size())
          l2.bias.data = l1.bias.data.view(l2.bias.size())
                

In [0]:

model = CNN().to(device) 
loss_func = nn.CrossEntropyLoss() 
optimizer = th.optim.Adam(model.parameters())
L, A, epochs = [], [], 10

NameError: ignored

In [23]:
"""
Define training procedure for network.
"""
model = CNN().to(device) 
model =model.float


         # Create an instance of our class, with
                                             # relu activation function. The .to(device)
                                             # moved the model from the cpu to gpu.

loss_func = nn.CrossEntropyLoss()            # Create cross entropy loss function.
optimizer = th.optim.Adam(model.parameters())# Create optimizer for training network.
                                             # The (model.parameters()) is passing the trainable
                                             # paramters of the model to the optmizer

L, A, epochs = [], [], 5                     # Lists for storing loss and accuracy
                                             # and the number of epochs to train the model.

for epoch in range(epochs):                  # Loop through 5 training epochs.

  for x, y in zip(a,b):  # Looping through all training samples.

    x = th.tensor(x, dtype=float)
    y = th.tensor(y, dtype=float)

    x, y = x.to(device), y.to(device)    # Move data to gpu-
    model.train()                     # Put model in training mode. This is important
                                      # if you have operations that behave differently
                                      # during training versus testing.

    optimizer.zero_grad()             # Pytorch accumulates gradients each time data is
                                      # passed through the network. The ".zero_grad()"
                                      # removes gradients calculated for preivous batches.

    output = model(x)             # Pass data through model
    loss = loss_func(output, y)   # Calculate loss.
    loss.backward()                   # Calculate gradients.
    optimizer.step()                  # Perform gradient descent, i.e. update parameters.

    L.append(loss.item())             # Store loss for plotting.

  for x, y in zip(c,d):  # Looping through all training samples.

    x = th.tensor(x, dtype=float)
    y = th.tensor(y, dtype=float)

    x, y = x.to(device), y.to(device)   # Move samples to gpu.
    model.eval()                  # Put model in evaluation mode. Again, if
                                  # you have operations that behave differently
                                  # during training versus testing.

    A.append(model.accuracy(x, y))  # Calculate accuracy

    # Plotting
    print('Epoch number: {}'.format(epoch))
    print('Cost: {}'.format(L[-1]))
    print('Acc: {}'.format(A[-1]))

    # Colab function for clearing the print log every fifth epoch.
    if epoch % 5 == 0 and epoch != 0:
        clear_output()

plt.figure(1)
plt.plot(L)
plt.show()

AttributeError: ignored